Note: this is a Python notebook, but it is only being used for markdown and a log of shell commands right now.

# Original Post-Processing Workflow

### R native data converted to TIFFs
On ROGER:  
`/projects/skope/PaleoCAR_RUN/OUTPUT/
                                 |---GDD_may_sept_demosaic
                                 |---PPT_annual_demosaic
                                 |---PPT_may_sept_demosaic
                                 |---PPT_water_year_demosaic`

`.rds` files are serialized R data in 1 degree tiles  
We need to ask Kyle how they were converted into TIFFs, since that was done outside of ROGER  
`.tif` are in 1 degree tiles, 2000 years stacked as bands

TIFFs are also in `/projects/skope/web/in/{data_type}/original/`  
(where {data_type} can also be replaced with the GDD and PPT data layers listed above)

#### [`split.pl`](https://github.com/openskope/paleocar-browser/blob/master/src/main/scripts/split.pl) (Adam)  
```
# This script takes all of Kyle's geotiffs for a given source and  
# (a) merges them into a single file  
# gdal_retile.py -ps 5000 5000 -targetDir .. "*.merge*.tif"  
# (b) splits out each band into a file  
# (c) colorizes the files (ramp.txt)  
# (d) creates the tiles for the web```

This script doesn't actually merge the 1 degree tiles, but the below cell shows a method for that

### Combine into single TIFF  
`
$ cd /projects/skope/in/
$ gdalbuildvrt GDD_may_sept_demosaic/GDD_may_sept_demosaic.vrt GDD_may_sept_demosaic/original/*.tif  
$ gdal_translate -of GTiff -co "BIGTIFF=YES" -co "COMPRESS=LZW" GDD_may_sept_demosaic/GDD_may_sept_demosaic.vrt GDD_may_sept_demosaic/GDD_may_sept_demosaic.tif `

### Separate bands into TIFF for each year
`split.pl` can handle this. It exports one band at a time from the stacked TIFF that coveres the whole output area into its own file. For example:  
`gdal_translate -b 1000 GDD_may_sept_demosaic.tif bands/GDD_may_sept_demosaic-1000.tif`

### Colorize bands with `gdaldem`

`split.pl` can handle this. It applies the colors ramp in ramp.txt to one year file at a time. For example:  
`gdaldem color-relief bands/GDD_may_sept_demosaic-1000.tif ../ramp.txt color/GDD_may_sept_demosaic-1000-color.tif`

#### Color Ramp [ramp.txt](https://github.com/openskope/paleocar-browser/blob/master/src/main/scripts/ramp.txt)
```nv 255 255 255 255
0 46 154 88
1600 251 255 128
3200 224 108 31
4800 200 55 55
6000 215 244 244```

![](tms_overlay_wide.png)

# Fixing existing TIFFs
Johnathan used desktop GIS to create a TIFF over the problematic area with all -32768 values, which is the value used for nodata.

Simply adding it to an uncolorized TIFF will make the 1 degree square chunk in the southwest corner will correct the values to make everything outside of the US states nodata.
However, make sure that those values aren't set as nodata in this fix tiff - they need to be considered real values for the addition to work.  

##### Unset nodata value in fixing TIFF
`$ cd /projects/skope/web/in/fixtiles`  
`$ gdal_edit.py -unsetnodata fix6.tif`  

##### Do the addition  
`$ gdal_calc.py -A ../GDD_may_sept_demosaic/band/GDD_may_sept_demosaic-1000.tif -B fix6.tif --outfile=GDD_1000_fix6.tif --calc="A+B"`

##### Re-colorizing fixed TIFF
`gdaldem color-relief GDD_1000_fix6.tif ramp.txt GDD_1000_color_LZW.tif -co "COMPRESS=DEFLATE"`

##### For list of TMS tiles
`$ for i in $(seq 0 12); do ./globalmaptiles.py $i 31 -115 32 -114 >> tiles.txt; done  
$ grep TileMapService tiles.txt  `

You can also use that script to determine the total number of tiles for the whole area of interest over the 4 corners states:  
`  
$ for i in $(seq 0 12); do ./globalmaptiles.py $i 30.9958330 -114.9958330 42.9958330 -101.9958330 >> fullextent0-12.txt; done  
$ grep TileMapService fullextent0-12.txt | wc -l
34400
`

We probably don't need to generate tiles down to the 12th zoom level. For 2000 years, that would mean 68,820,000 tiles.

#### To do: modify the parallel gdal2tiles script to run only the necessary tiles that need to be replaced
That script uses nested loops for zoom, columns, and rows.  Quickest thing would be to put a conditional check if the current tile is part of a list of tiles to calculate.  